In [1]:
import sys
sys.path.append("../")

In [12]:
PROJECT = 'kaggle-hackathon-471317'
BQ_LOC = "US"
COLL = "default_collection"
DS_ID = "poi-locations-ds"
DS_DISPLAY_NAME = "POI Locations Datastore"

In [3]:
from app.utils.datastore_manager import *

In [4]:
from typing import List

from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine_v1 as discoveryengine

In [ ]:
# 1) Create
ds = create_datastore(
    project_id=PROJECT,
    location=LOC,
    data_store_id=DS_ID,
    display_name=DS_DISPLAY_NAME,
    industry_vertical=discoveryengine.IndustryVertical.GENERIC,
    solution_types=(discoveryengine.SolutionType.SOLUTION_TYPE_SEARCH,),
    collection_id=COLL,
)
print("Created:", ds.name)

In [ ]:
DS = "geo_intents"
TBL = DS_ID
LOC = "global"

In [9]:
from app.utils.bigquery_manager import *

In [13]:
# 1) Dataset lifecycle
create_dataset(PROJECT, DS, location=BQ_LOC, description="Geo Intents Analytics")
print([d.dataset_id for d in list_datasets(PROJECT)])
ds = get_dataset(PROJECT, DS)


Conflict: 409 POST https://bigquery.googleapis.com/bigquery/v2/projects/kaggle-hackathon-471317/datasets?prettyPrint=false: Already Exists: Dataset kaggle-hackathon-471317:geo_intents

In [ ]:
job = run_query(
    PROJECT,
    """
SELECT
  id as _id,
  geometry,
  bbox,
  names.primary AS name,
  categories.primary AS category,
  brand.names.`primary` AS brand,
IF
  (ARRAY_LENGTH(addresses.`list`)>0, addresses.`list`[SAFE_OFFSET(0)].element.freeform, NULL) AS street_address,
IF
  (ARRAY_LENGTH(addresses.`list`)>0, addresses.`list`[SAFE_OFFSET(0)].element.locality, NULL) AS locality,
IF
  (ARRAY_LENGTH(addresses.`list`)>0, addresses.`list`[SAFE_OFFSET(0)].element.postcode, NULL) AS postcode,
IF
  (ARRAY_LENGTH(addresses.`list`)>0, addresses.`list`[SAFE_OFFSET(0)].element.region, NULL) AS region,
IF
  (ARRAY_LENGTH(addresses.`list`)>0, addresses.`list`[SAFE_OFFSET(0)].element.country, NULL) AS country,
FROM
  bigquery-public-data.overture_maps.place
WHERE
  addresses.list[
OFFSET
  (0)].element.region="CA"
  AND categories.primary IN ( # Core Coffee Competitors Categories 'coffee_shop',
    'coffee_roastery',
    'cafe',
    'hong_kong_style_cafe',
    'bubble_tea',
    'tea_room',
    'bakery',
    'donuts',
    'afghan_restaurant',
    'african_restaurant',
    'american_restaurant',
    'arabian_restaurant',
    'argentine_restaurant',
    'armenian_restaurant',
    'asian_fusion_restaurant',
    'asian_grocery_store',
    'asian_restaurant',
    'australian_restaurant',
    'austrian_restaurant',
    'bagel_restaurant',
    'bakery',
    'bangladeshi_restaurant',
    'bar_and_grill_restaurant',
    'barbecue_restaurant',
    'basque_restaurant',
    'belgian_restaurant',
    'brazilian_restaurant',
    'breakfast_and_brunch_restaurant',
    'british_restaurant',
    'buffet_restaurant',
    'burger_restaurant',
    'burmese_restaurant',
    'cajun_and_creole_restaurant',
    'cambodian_restaurant',
    'canadian_restaurant',
    'caribbean_restaurant',
    'cheesesteak_restaurant',
    'chicken_restaurant',
    'chicken_wings_restaurant',
    'chilean_restaurant',
    'chinese_restaurant',
    'colombian_restaurant',
    'comfort_food_restaurant',
    'convenience_store',
    'cuban_restaurant',
    'delicatessen',
    'dim_sum_restaurant',
    'diy_foods_restaurant',
    'dominican_restaurant',
    'donuts',
    'dumpling_restaurant',
    'eastern_european_restaurant',
    'ecuadorian_restaurant',
    'egyptian_restaurant',
    'ethiopian_restaurant',
    'filipino_restaurant',
    'french_restaurant',
    'frozen_yogurt_shop',
    'fusion_restaurant',
    'gastropub_restaurant',
    'georgian_restaurant',
    'german_restaurant',
    'gluten_free_restaurant',
    'greek_restaurant',
    'grocery_store',
    'guatemalan_restaurant',
    'haitian_restaurant',
    'hawaiian_restaurant',
    'himalayan_restaurant',
    'honduran_restaurant',
    'hot_dog_restaurant',
    'hungarian_restaurant',
    'ice_cream_shop',
    'indian_restaurant',
    'indonesian_restaurant',
    'irish_restaurant',
    'italian_restaurant',
    'jamaican_restaurant',
    'japanese_restaurant',
    'juice_shop',
    'korean_restaurant',
    'kosher_restaurant',
    'laotian_restaurant',
    'latin_american_restaurant',
    'lebanese_restaurant',
    'malaysian_restaurant',
    'mediterranean_restaurant',
    'mexican_restaurant',
    'middle_eastern_restaurant',
    'modern_european_restaurant',
    'mongolian_restaurant',
    'moroccan_restaurant',
    'nepalese_restaurant',
    'new_american_restaurant',
    'nicaraguan_restaurant',
    'pakistani_restaurant',
    'pan_asian_restaurant',
    'peruvian_restaurant',
    'philippine_restaurant',
    'pizza_restaurant',
    'polish_restaurant',
    'portuguese_restaurant',
    'puerto_rican_restaurant',
    'ramen_restaurant',
    'raw_food_restaurant',
    'romanian_restaurant',
    'russian_restaurant',
    'salad_restaurant',
    'salvadoran_restaurant',
    'sandwich_shop',
    'scandinavian_restaurant',
    'seafood_restaurant',
    'singaporean_restaurant',
    'slovak_restaurant',
    'soul_food_restaurant',
    'south_african_restaurant',
    'south_american_restaurant',
    'south_asian_restaurant',
    'southeast_asian_restaurant',
    'southern_restaurant',
    'spanish_restaurant',
    'steakhouse_restaurant',
    'street_food_restaurant',
    'supermarket',
    'sushi_restaurant',
    'syrian_restaurant',
    'taiwanese_restaurant',
    'tapas_restaurant',
    'tex_mex_restaurant',
    'thai_restaurant',
    'turkish_restaurant',
    'ukrainian_restaurant',
    'uzbek_restaurant',
    'vegan_restaurant',
    'vegetarian_restaurant',
    'venezuelan_restaurant',
    'vietnamese_restaurant',
    'waffle_shop',
    'west_african_restaurant',
    'wine_bar_restaurant',
    'wrap_restaurant',
    'yemenite_restaurant' )""",
    location=BQ_LOC,
    destination_table=f"{PROJECT}.{DS}.{TBL}",
    write_disposition=WriteDisposition.WRITE_TRUNCATE,
)

In [15]:
import_documents_from_bigquery(
    project_id=PROJECT,
    location=LOC,
    data_store_id=DS_ID,
    bq_project_id=PROJECT,
    bq_dataset=DS,
    bq_table=TBL,
    data_schema="custom",   # typical for structured/JSON
    branch_id="default_branch",
    collection_id=COLL,
)


In [7]:
print(get_datastore(PROJECT, LOC, DS_ID))
for ds in list_datastores(PROJECT, LOC, COLL):
    print(ds.name, ds.display_name)

name: "projects/168027251845/locations/global/collections/default_collection/dataStores/poi-locations-ds"
display_name: "POI Locations Datastore"
industry_vertical: GENERIC
create_time {
  seconds: 1757296842
  nanos: 244138000
}
solution_types: SOLUTION_TYPE_SEARCH
content_config: NO_CONTENT
default_schema_id: "default_schema"

projects/168027251845/locations/global/collections/default_collection/dataStores/poi-locations-ds POI Locations Datastore
